In [1]:
from API_call import API_call
from functions import get_gamePks,read_gamePks
from datetime import datetime as dt
import os,re,csv
from os import walk
import statsapi as mlb
import pandas as pd

In [2]:
get_gamePks([str(x) for x in range(2008,2020)])   

gamePks=read_gamePks()

In [3]:
gamePks['2019'][500]

'566385'

In [4]:
call = API_call('566385')

In [5]:
call.game

{'pk': 566385,
 'type': 'R',
 'doubleHeader': 'N',
 'id': '2019/04/26/texmlb-seamlb-1',
 'gamedayType': 'P',
 'tiebreaker': 'N',
 'gameNumber': 1,
 'calendarEventID': '14-566385-2019-04-26',
 'season': '2019',
 'dateTime': datetime.datetime(2019, 4, 27, 2, 10),
 'originalDate': datetime.date(2019, 4, 26),
 'condition': 'Cloudy',
 'temp': '56',
 'wind': '5 mph, In From CF',
 'tz': 'PDT',
 'detailedState': 'Final',
 'homeTeam_id': 136,
 'awayTeam_id': 140,
 'venue_id': 680,
 'home_probablePitcher': 579328,
 'away_probablePitcher': 571946}

In [6]:
plays = pd.DataFrame.from_records(call.plays)

In [7]:
plays.columns

Index(['result_type', 'result_event', 'result_eventType', 'result_description',
       'result_rbi', 'result_awayScore', 'result_homeScore',
       'about_atBatIndex', 'about_halfInning', 'about_isTopInning',
       'about_inning', 'about_startTime', 'about_endTime', 'about_isComplete',
       'about_isScoringPlay', 'about_hasReview', 'about_hasOut',
       'about_captivatingIndex', 'count_balls', 'count_strikes', 'count_outs',
       'gamePk'],
      dtype='object')

In [8]:
pitches = pd.DataFrame.from_records(call.pitches)

In [9]:
pitches.columns

Index(['index', 'pfxId', 'playId', 'pitchNumber', 'startTime', 'endTime',
       'isPitch', 'type', 'atBatIndex', 'playEndTime', 'details_description',
       'details_code', 'details_ballColor', 'details_trailColor',
       'details_isInPlay', 'details_isStrike', 'details_isBall',
       'details_hasReview', 'count_balls', 'count_strikes',
       'pitchData_startSpeed', 'pitchData_endSpeed', 'pitchData_strikeZoneTop',
       'pitchData_strikeZoneBottom', 'pitchData_zone',
       'pitchData_typeConfidence', 'pitchData_plateTime',
       'pitchData_extension', 'details_call_code', 'details_call_description',
       'details_type_code', 'details_type_description',
       'pitchData_coordinates_aY', 'pitchData_coordinates_aZ',
       'pitchData_coordinates_pfxX', 'pitchData_coordinates_pfxZ',
       'pitchData_coordinates_pX', 'pitchData_coordinates_pZ',
       'pitchData_coordinates_vX0', 'pitchData_coordinates_vY0',
       'pitchData_coordinates_vZ0', 'pitchData_coordinates_x',
       '

In [90]:
hitDataColumns = [x for x in pitches.columns if 'hitData' in x]

In [91]:
hitData = pitches[hitDataColumns].dropna(axis=0,how='all')

In [92]:
hitData.shape

(53, 8)

In [93]:
pitchDataColumns = [x for x in pitches.columns if 'pitchData' in x]
pitchData = pitches[pitchDataColumns].dropna(axis=0,how='all')
pitchData.shape

(396, 28)

In [115]:
test_pks = gamePks['2019'][::20]
pitches = []
for pk in test_pks:
    api_call = API_call(pk)
    pitches.extend(api_call.pitches)

In [116]:
pitches = pd.DataFrame.from_records(pitches)

In [139]:
pitches[hitDataColumns].shape

(40665, 8)

In [138]:
pitches[pitchDataColumns].shape

(40665, 28)

In [137]:
pitches.shape

(40665, 63)

In [136]:
pitches[['atBatIndex','playEndTime','index']].shape==pitches[['atBatIndex','playEndTime','index']].drop_duplicates().shape

True

In [142]:
pitchDataColumns

['pitchData_startSpeed',
 'pitchData_endSpeed',
 'pitchData_strikeZoneTop',
 'pitchData_strikeZoneBottom',
 'pitchData_zone',
 'pitchData_typeConfidence',
 'pitchData_plateTime',
 'pitchData_extension',
 'pitchData_coordinates_aY',
 'pitchData_coordinates_aZ',
 'pitchData_coordinates_pfxX',
 'pitchData_coordinates_pfxZ',
 'pitchData_coordinates_pX',
 'pitchData_coordinates_pZ',
 'pitchData_coordinates_vX0',
 'pitchData_coordinates_vY0',
 'pitchData_coordinates_vZ0',
 'pitchData_coordinates_x',
 'pitchData_coordinates_y',
 'pitchData_coordinates_x0',
 'pitchData_coordinates_y0',
 'pitchData_coordinates_z0',
 'pitchData_coordinates_aX',
 'pitchData_breaks_breakAngle',
 'pitchData_breaks_breakLength',
 'pitchData_breaks_breakY',
 'pitchData_breaks_spinRate',
 'pitchData_breaks_spinDirection']

In [141]:
pitches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40665 entries, 0 to 40664
Data columns (total 63 columns):
index                             40665 non-null int64
playId                            40060 non-null object
pitchNumber                       39748 non-null float64
startTime                         39748 non-null object
endTime                           39748 non-null object
isPitch                           40665 non-null bool
type                              40665 non-null object
atBatIndex                        40665 non-null int64
playEndTime                       40665 non-null object
details_description               40665 non-null object
details_code                      40665 non-null object
details_ballColor                 39748 non-null object
details_isInPlay                  39748 non-null object
details_isStrike                  39748 non-null object
details_isBall                    39748 non-null object
details_hasReview                 40665 non-null bool


In [13]:
details = [x for x in pitches.columns if 'details' in x]

pitches[(pitches['pfxId'].isna())&pitches['isPitch']==True][details]

In [16]:
pd.to_datetime(plays['about_startTime'])

0    2019-04-27 00:55:22+00:00
1    2019-04-27 02:11:26+00:00
2    2019-04-27 02:13:18+00:00
3    2019-04-27 02:16:28+00:00
4    2019-04-27 02:18:21+00:00
                ...           
87   2019-04-27 06:01:59+00:00
88   2019-04-27 06:04:55+00:00
89   2019-04-27 06:07:23+00:00
90   2019-04-27 06:10:40+00:00
91   2019-04-27 06:14:08+00:00
Name: about_startTime, Length: 92, dtype: datetime64[ns, UTC]

In [17]:
plays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 22 columns):
result_type               92 non-null object
result_event              92 non-null object
result_eventType          92 non-null object
result_description        92 non-null object
result_rbi                92 non-null int64
result_awayScore          92 non-null int64
result_homeScore          92 non-null int64
about_atBatIndex          92 non-null int64
about_halfInning          92 non-null object
about_isTopInning         92 non-null bool
about_inning              92 non-null int64
about_startTime           92 non-null object
about_endTime             92 non-null object
about_isComplete          92 non-null bool
about_isScoringPlay       92 non-null bool
about_hasReview           92 non-null bool
about_hasOut              92 non-null bool
about_captivatingIndex    92 non-null int64
count_balls               92 non-null int64
count_strikes             92 non-null int64
count_outs     

In [44]:
from database import *

In [50]:
db = MyDatabase('sqlite',dbname='mlb.db')

Engine(sqlite://///Users/schlinkertc/code/MLB/mlb.db)


In [51]:
Base.metadata.create_all(db.db_engine)

In [55]:
game = Game(call.game)

In [54]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db.db_engine)
session = Session()

In [56]:
session.add(game)

In [57]:
session.commit()

In [58]:
session.query(Game).all()

[<Game(pk='566385',id='2019/04/26/texmlb-seamlb-1')>]

In [60]:
session.add_all([Play(play) for play in call.plays])
session.commit()

In [62]:
len(session.query(Play).all())==len(call.plays)

True

In [65]:
pitches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410 entries, 0 to 409
Data columns (total 63 columns):
index                             410 non-null int64
pfxId                             393 non-null object
playId                            407 non-null object
pitchNumber                       396 non-null float64
startTime                         396 non-null object
endTime                           396 non-null object
isPitch                           410 non-null bool
type                              410 non-null object
atBatIndex                        410 non-null int64
playEndTime                       410 non-null object
details_description               410 non-null object
details_code                      410 non-null object
details_ballColor                 396 non-null object
details_trailColor                392 non-null object
details_isInPlay                  396 non-null object
details_isStrike                  396 non-null object
details_isBall                    

ValueError: Must pass DataFrame with boolean values only